Copyright © The University of Edinburgh, 2024.

Development has been supported by GSK.

# Setup
Load CMAP data from Phenonaut objects and define directories

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import pandas as pd
import leakproofcmap
working_dir=Path("working_dir")
packaged_dataset_dir=Path("/local_scratch/data/phenonaut_datasets/cmap/")
pickle_dir=(working_dir/Path("pickles")).resolve()
split_data=(working_dir/Path("split_data")).resolve()
if not split_data.exists():
    split_data.mkdir(parents=True)

phe=leakproofcmap.get_cmap_phenonaut_object(phenonaut_packaged_dataset_dir=packaged_dataset_dir,working_dir=working_dir,pickle_dir=pickle_dir)


# Write out MOA and pert_iname counts

This is purely for accounting and seeing how many of each we have

Writes 'moa_counts.tsv' and 'pert_iname_counts.tsv' to working_dir/split_data


In [ ]:
tmp_df=phe.df.moa.value_counts().to_frame("count")
tmp_df.index.name = 'moa'
tmp_df.to_csv(split_data/"moa_counts.tsv", sep="\t")
del tmp_df
tmp_df=phe.df.pert_iname.value_counts().to_frame("counts")
tmp_df.index.name = 'pert_iname'
tmp_df.to_csv(split_data/"pert_iname_counts.tsv", sep="\t")
del tmp_df

# Display the number of cell lines assessed against each MOA


In [ ]:
from tqdm import tqdm
moas_in_cell_line_counts = {moa:len(phe.df.query("moa == @moa")['cell_id'].unique()) for moa in tqdm(phe.df.moa.unique())}
print({k:v for k,v in sorted(moas_in_cell_line_counts.items(), key=lambda x: -x[1])})

# Generate cell line splits

Write out splits based on cell line response to median moa perturbations

In [ ]:
import phenonaut
from phenonaut.transforms.imputers import KNNImputer

cell_line_to_moa_median_responses_df_file=split_data/"cell_line_to_moa_median_imputed.tsv"

if not cell_line_to_moa_median_responses_df_file.exists():
    print("Doesnt exists")
    cell_line_to_moa_df_with_nans_file=split_data/"cell_line_to_moa_median_with_nans.tsv"
    if not cell_line_to_moa_df_with_nans_file.exists():
        counts_df=phe.df.groupby(["cell_id", "moa"])['DDR1'].count().reset_index().rename(columns={'DDR1':'count'}).pivot(index='cell_id', columns=['moa'], values='count')
        counts_df.fillna(0).astype(int).to_csv(split_data/"cell_line_to_moa_count.tsv", sep="\t")

        cell_line_to_moa_df=pd.DataFrame(columns=[f"{moa}_{f}" for moa in counts_df.columns for f in phe.ds.features], index=counts_df.index)
        for cell_line in counts_df.index:
            groups=phe.df.query("cell_id==@cell_line").groupby('moa')
            for moa, g_df in groups:
                cell_line_to_moa_df.loc[cell_line, [f"{moa}_{f}" for f in phe.ds.features]]=g_df[phe.ds.features].median().values
        cell_line_to_moa_df.to_csv(cell_line_to_moa_df_with_nans_file, sep="\t")
    else:
        print("Found file, reading")
        cell_line_to_moa_df=pd.read_csv(cell_line_to_moa_df_with_nans_file, sep="\t", index_col=[0])
    counts_df=phe.df.groupby(["cell_id", "moa"])['DDR1'].count().reset_index().rename(columns={'DDR1':'count'}).pivot(index='cell_id', columns=['moa'], values='count')
    phe_cell_line_to_moa_response=phenonaut.Phenonaut(phenonaut.data.Dataset("cell_line_to_moa_response", input_file_path_or_df=cell_line_to_moa_df, features=[f"{m}_{f}" for f in phe.ds.features for m in counts_df.columns]))
    imputer=KNNImputer()
    print("Running imputation")
    imputer(phe_cell_line_to_moa_response.ds)
    tmp_df=phe_cell_line_to_moa_response.df[phe_cell_line_to_moa_response.ds.features]
    tmp_df.to_csv(cell_line_to_moa_median_responses_df_file, sep="\t")
    del tmp_df
    del phe_cell_line_to_moa_response

cell_line_to_moa_median_responses_df=pd.read_csv(cell_line_to_moa_median_responses_df_file, sep="\t", index_col=[0])


Define function to perform diverse splits


In [ ]:
from numpy.random import default_rng
from scipy.spatial.distance import pdist, squareform

import numpy as np

def distribute_data_into_splits(response_df:pd.DataFrame, n_splits:int=5, random_state:int=7)->pd.DataFrame:
    np_rng=np.random.default_rng(random_state)
    splits=[[] for _ in range(n_splits)]
    distances=pd.DataFrame(squareform(pdist(response_df, metric='euclidean')), index=response_df.index, columns=response_df.index)
    distances.values[np.eye(len(distances), dtype=bool)]=np.nan
    # Add most diverse to splits
    for split, md in zip(splits, np_rng.permutation(distances.sum(axis=0).sort_values(ascending=False).index[:n_splits])):
        split.append(md)
        distances.loc[:,md]=np.nan

    # Add the first most similar sample to the split (dealing with closest to split[:, 0])
    for split in splits:
        min_dist_index=np.nanargmin(distances.loc[split[0]].values)
        split.append(distances.index.tolist()[min_dist_index])
        distances.loc[:, distances.index.tolist()[min_dist_index]]=np.nan

    cur_split_idx=0
    # As above, add most similar to any of the others in the split
    while len([item for split in splits for item in split])<len(response_df):
        split_dists=distances.loc[splits[cur_split_idx],:]
        min_dist_idx=np.unravel_index(np.nanargmin(distances.loc[splits[cur_split_idx],:]),split_dists.shape)
        splits[cur_split_idx].append(split_dists.columns.tolist()[min_dist_idx[1]])
        distances.loc[:,split_dists.columns.tolist()[min_dist_idx[1]]]=np.nan
        cur_split_idx+=1
        if cur_split_idx==len(splits):
            cur_split_idx=0
    return pd.DataFrame(splits, index=[f'split_{n}' for n in range(1, n_splits+1)])


Perform splitting and write to file


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=None))])

cell_line_to_moa_median_responses_df_stdscale_pca=pd.DataFrame(pipeline.fit_transform(cell_line_to_moa_median_responses_df), index=cell_line_to_moa_median_responses_df.index)

splits_by_cell_line_based_on_moa_df=distribute_data_into_splits(cell_line_to_moa_median_responses_df_stdscale_pca, 5)
splits_by_cell_line_based_on_moa_df.to_csv(split_data/"cmap_splits_by_cell_line_based_on_moa.tsv", sep="\t")

### Generate MOA splits

In [ ]:
# Gather info on cell line to (MOA), define cell line splits by MOA

import phenonaut
from phenonaut.transforms.imputers import KNNImputer, RFImputer

moa_to_cell_line_median_responses_df_file=split_data/"moa_to_cell_line_median_imputed.tsv"
if not moa_to_cell_line_median_responses_df_file.exists():
    moa_to_cell_line_df_with_nans_file=split_data/"moa_to_cell_line_median_with_nans.tsv"
    if not moa_to_cell_line_df_with_nans_file.exists():
        print(f"Generating {moa_to_cell_line_median_responses_df_file}")
        counts_df=phe.df.groupby(["cell_id", "moa"])['DDR1'].count().reset_index().rename(columns={'DDR1':'count'}).pivot(index='moa', columns=['cell_id'], values='count')
        counts_df.fillna(0).astype(int).to_csv(split_data/"moa_to_cell_line_count.tsv", sep="\t")

        moa_to_cell_line_df=pd.DataFrame(columns=[f"{cl}_{f}" for cl in counts_df.columns for f in phe.ds.features], index=counts_df.index)
        for moa in counts_df.index:
            groups=phe.df.query("moa==@moa").groupby('cell_id')
            for cell_line, g_df in groups:
                moa_to_cell_line_df.loc[moa, [f"{cell_line}_{f}" for f in phe.ds.features]]=g_df[phe.ds.features].median().values
            moa_to_cell_line_df.dropna(axis=1, how='all')
        moa_to_cell_line_df.to_csv(moa_to_cell_line_df_with_nans_file, sep="\t")
    else:
        print("Found file, reading")
        moa_to_cell_line_df=pd.read_csv(moa_to_cell_line_df_with_nans_file, sep="\t", index_col=[0])
    phe_cell_line_to_moa_response=phenonaut.Phenonaut(phenonaut.data.Dataset("moa_to_cell_line_response", input_file_path_or_df=moa_to_cell_line_df, features=[f"{cl}_{f}" for cl in counts_df.columns for f in phe.ds.features]))
    imputer=KNNImputer()
    print("Running imputation")
    imputer(phe_cell_line_to_moa_response.ds)
    tmp_df=phe_cell_line_to_moa_response.df[phe_cell_line_to_moa_response.ds.features]
    tmp_df.to_csv(moa_to_cell_line_median_responses_df_file, sep="\t")
moa_to_cell_line_median_responses_df=pd.read_csv(moa_to_cell_line_median_responses_df_file, sep="\t", index_col=[0])



Perform splitting and write to file


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
print(moa_to_cell_line_median_responses_df.shape)
moa_to_cell_line_median_responses_df_no_dmso=moa_to_cell_line_median_responses_df.drop(index='control vehicle')
print(moa_to_cell_line_median_responses_df_no_dmso.shape)
pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=None))])

moa_to_cell_line_median_responses_df_stdscale_pca=pd.DataFrame(pipeline.fit_transform(moa_to_cell_line_median_responses_df_no_dmso), index=moa_to_cell_line_median_responses_df_no_dmso.index)

splits_by_moa_based_on_cell_line_df=distribute_data_into_splits(moa_to_cell_line_median_responses_df_stdscale_pca, 5)
splits_by_moa_based_on_cell_line_df.to_csv(split_data/"cmap_splits_by_moa_based_on_cell_line.tsv", sep="\t")

# Write split data

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import pandas as pd
import leakproofcmap

working_dir=Path("working_dir")
packaged_dataset_dir=Path("/local_scratch/data/phenonaut_datasets/cmap/")
pickle_dir=(working_dir/Path("pickles")).resolve()
split_data=(working_dir/Path("split_data")).resolve()
if not split_data.exists():
    split_data.mkdir(parents=True)

phe=leakproofcmap.get_cmap_phenonaut_object(phenonaut_packaged_dataset_dir=packaged_dataset_dir,working_dir=working_dir,pickle_dir=pickle_dir)

split_data=working_dir/"split_data"
if not split_data.exists():
    split_data.mkdir(parents=True)
for cell_id_split_i in range(1, 5 + 1):
    for moa_split_i in range(1, 5 + 1):
        name=f"cellidsplit{cell_id_split_i}_moasplit{moa_split_i}"
        split = leakproofcmap.CMAPSplit.from_splits_tsv(
            cell_id_split_file=split_data / "cmap_splits_by_cell_line_based_on_moa.tsv",
            moa_split_file=split_data / "cmap_splits_by_moa_based_on_cell_line.tsv",
            cell_line_split_number=cell_id_split_i,
            moa_split_number=moa_split_i,df=phe.df, features=phe.ds.features,
            name=name,
            np_rng_seed=42,
            is_scalar_and_pca_target=True if (cell_id_split_i==1 and moa_split_i==1) else False,
        )
        split.save(split_data/f"cmap_split_{name}.json")


# Write split cpd and moa counts

In [ ]:

ds_counts_df=pd.DataFrame(columns=["n_records","unique_moas","unique_compounds"], index=pd.MultiIndex.from_product([range(1,6), range(1,6), range(1,6), ["fold_type"]], names=['cli', 'moai', 'fold_i', 'fold_type'])).sort_index()

for cell_line_i in range(1,6):
    for moa_i in range(1,6):
        for fold_i in range(1,6):
            split_object=leakproofcmap.CMAPSplit.load(split_data/Path(f"cmap_split_cellidsplit{cell_line_i}_moasplit{moa_i}.json"))
            df, _ = split_object.get_df(phe.df, phe.ds.features, False, False, "train", fold_number=fold_i)
            ds_counts_df.loc[(cell_line_i,moa_i, fold_i, "train"),"n_records"]=df.shape[0]
            ds_counts_df=ds_counts_df.sort_index()

            ds_counts_df.loc[(cell_line_i,moa_i, fold_i, "train"),"unique_moas"]=len(df.moa.unique())
            ds_counts_df=ds_counts_df.sort_index()
            ds_counts_df.loc[(cell_line_i,moa_i, fold_i, "train"),"unique_compounds"]=len(df.pert_iname.unique())
            ds_counts_df=ds_counts_df.sort_index()
            df, _ = split_object.get_df(phe.df, phe.ds.features, False, False, "val", fold_number=fold_i)
            ds_counts_df.loc[(cell_line_i,moa_i, fold_i, "val"),"n_records"]=df.shape[0]
            ds_counts_df=ds_counts_df.sort_index()
            ds_counts_df.loc[(cell_line_i,moa_i, fold_i, "val"),"unique_moas"]=len(df.moa.unique())
            ds_counts_df=ds_counts_df.sort_index()
            ds_counts_df.loc[(cell_line_i,moa_i, fold_i, "val"),"unique_compounds"]=len(df.pert_iname.unique())
        df, _ = split_object.get_df(phe.df, phe.ds.features, False, False, "test")
        ds_counts_df.loc[(cell_line_i,moa_i, "NA", "test"),"n_records"]=df.shape[0]
        ds_counts_df=ds_counts_df.sort_index()
        ds_counts_df.loc[(cell_line_i,moa_i, "NA", "test"),"unique_moas"]=len(df.moa.unique())
        ds_counts_df=ds_counts_df.sort_index()
        ds_counts_df.loc[(cell_line_i,moa_i, "NA", "test"),"unique_compounds"]=len(df.pert_iname.unique())
        ds_counts_df=ds_counts_df.sort_index()
ds_counts_df=ds_counts_df.dropna(how='any', axis=0)
print(ds_counts_df)
ds_counts_df.to_csv(split_data/"split_info.tsv", sep="\t")
ds_counts_df.to_csv(split_data/"split_info.csv")